In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop unnecessary columns (EIN and NAME)
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [4]:
unique_counts = application_df.nunique()
unique_counts

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
app_type_counts = application_df['APPLICATION_TYPE'].value_counts()
app_type_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# One-hot encode categorical columns
application_df_encoded = pd.get_dummies(application_df, columns=['APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT', 'SPECIAL_CONSIDERATIONS'])

In [10]:
# Drop the original categorical columns
columns_to_drop = ['APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T12', 'APPLICATION_TYPE_T13', 'APPLICATION_TYPE_T14', 'APPLICATION_TYPE_T15',
                   'APPLICATION_TYPE_T17', 'APPLICATION_TYPE_T19', 'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999', 'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M',
                   'INCOME_AMT_25000-99999', 'INCOME_AMT_50M+', 'INCOME_AMT_5M-10M', 'SPECIAL_CONSIDERATIONS_N', 'SPECIAL_CONSIDERATIONS_Y']

application_df_encoded = application_df_encoded.drop(columns_to_drop, axis=1)

application_df_encoded.head()



,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T2,APPLICATION_TYPE_T25,APPLICATION_TYPE_T29,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0
0,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
1,1,108590,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,1,1,0,0,0,1
3,1,6692,1,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0


In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the model
nn_model = Sequential()

# Add the first hidden layer
nn_model.add(Dense(units=32, input_dim=len(application_df_encoded.columns)-1, activation='relu'))

# Add additional hidden layers (optional)
nn_model.add(Dense(units=16, activation='relu'))

# Add the output layer
nn_model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Display the model summary
nn_model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                3200      
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 3745 (14.63 KB)
Trainable params: 3745 (14.63 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [13]:
# Split data into features (X) and target (y)
X = application_df_encoded.drop('IS_SUCCESSFUL', axis=1)
y = application_df_encoded['IS_SUCCESSFUL']

# Perform feature scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=42, stratify=y)


In [14]:
# Train the model
nn_model.fit(X_train, y_train, epochs=100, verbose=1, validation_data=(X_test, y_test))


Epoch 1/100
804/804 [==============================] - 4s 4ms/step - loss: 0.5928 - accuracy: 0.7086 - val_loss: 0.5792 - val_accuracy: 0.7060
Epoch 2/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5687 - accuracy: 0.7187 - val_loss: 0.5735 - val_accuracy: 0.7106
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5633 - accuracy: 0.7248 - val_loss: 0.5699 - val_accuracy: 0.7155
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5605 - accuracy: 0.7245 - val_loss: 0.5700 - val_accuracy: 0.7113
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5587 - accuracy: 0.7253 - val_loss: 0.5724 - val_accuracy: 0.7149
Epoch 6/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5580 - accuracy: 0.7255 - val_loss: 0.5672 - val_accuracy: 0.7115
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5573 - accuracy: 0.7261 - val_loss: 0.5667 - val_accuracy: 0.7097

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


268/268 - 1s - loss: 0.5746 - accuracy: 0.7201 - 503ms/epoch - 2ms/step
Loss: 0.5746006965637207, Accuracy: 0.7201166152954102


In [16]:
# Define an updated model
nn_model_updated = Sequential()
nn_model_updated.add(Dense(units=64, input_dim=len(application_df_encoded.columns)-1, activation='relu'))
nn_model_updated.add(Dense(units=32, activation='relu'))
nn_model_updated.add(Dense(units=16, activation='relu'))
nn_model_updated.add(Dense(units=1, activation='sigmoid'))

# Compile the updated model
nn_model_updated.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the updated model
nn_model_updated.fit(X_train, y_train, epochs=150, verbose=1, validation_data=(X_test, y_test))

# Evaluate the updated model
updated_model_loss, updated_model_accuracy = nn_model_updated.evaluate(X_test, y_test, verbose=2)
print(f"Updated Model Loss: {updated_model_loss}, Updated Model Accuracy: {updated_model_accuracy}")


Epoch 1/150
804/804 [==============================] - 3s 3ms/step - loss: 0.5844 - accuracy: 0.7121 - val_loss: 0.5723 - val_accuracy: 0.7106
Epoch 2/150
804/804 [==============================] - 3s 4ms/step - loss: 0.5622 - accuracy: 0.7217 - val_loss: 0.5715 - val_accuracy: 0.7129
Epoch 3/150
804/804 [==============================] - 3s 4ms/step - loss: 0.5591 - accuracy: 0.7247 - val_loss: 0.5725 - val_accuracy: 0.7117
Epoch 4/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5579 - accuracy: 0.7241 - val_loss: 0.5643 - val_accuracy: 0.7153
Epoch 5/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5567 - accuracy: 0.7257 - val_loss: 0.5633 - val_accuracy: 0.7124
Epoch 6/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5561 - accuracy: 0.7268 - val_loss: 0.5664 - val_accuracy: 0.7115
Epoch 7/150
804/804 [==============================] - 2s 3ms/step - loss: 0.5547 - accuracy: 0.7274 - val_loss: 0.5649 - val_accuracy: 0.7146